## VaR Monte Carlo with as many Tickers symbols as you want.

In [111]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import norm

In [112]:
years=15

endDate=dt.datetime.now()
startDate=endDate-dt.timedelta(days=365*years)

In [113]:
tickers=['TSLA','GOOGL','QQQ','VTI','AMD','AMZN'] ## Put as many as tickers as you want

In [114]:
## Next get adjusted closing price

adj_close_df=pd.DataFrame()

In [115]:
for i in tickers:
    data=yf.download(i,start=startDate,end=endDate)
    adj_close_df[i]=data['Adj Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [116]:
## We will get the log returns because those are consistent in many aspect than just returns. Secondly, take the first difference as they are not stationary.
log_returns=np.log(adj_close_df/adj_close_df.shift(1))


In [117]:
## Removing the first row because after the difference one gets NaN
log_returns=log_returns.dropna()

In [118]:
log_returns

,TSLA,GOOGL,QQQ,VTI,AMD,AMZN
Date,,,,,,
2010-06-30,-0.002515,-0.020708,-0.015335,-0.010974,-0.021622,0.005967
2010-07-01,-0.081723,-0.012347,-0.002813,-0.004386,0.009517,0.015439
2010-07-02,-0.134312,-0.006712,-0.002822,-0.004789,-0.030222,-0.016538
2010-07-06,-0.175470,-0.001100,0.003056,0.003067,-0.018298,0.008394
2010-07-07,-0.019431,0.031889,0.031426,0.032024,0.048520,0.030160
...,...,...,...,...,...,...
2023-09-25,0.008580,0.006581,0.004739,0.004017,0.012191,0.016514
2023-09-26,-0.011688,-0.019563,-0.015130,-0.014510,-0.014689,-0.041133
2023-09-27,-0.014940,0.015206,0.002341,0.001040,0.021750,0.000000


In [119]:
def expected_return(weights,log_returns):
    return np.sum(log_returns.mean()*weights)

In [120]:
def standard_deviation(weights,cov_matrix):
    variance=weights.T@cov_matrix@weights
    return np.sqrt(variance)


In [121]:
cov_matrix=log_returns.cov()
cov_matrix

,TSLA,GOOGL,QQQ,VTI,AMD,AMZN
TSLA,0.001285,0.000209,0.000235,0.000180,0.000399,0.000266
GOOGL,0.000209,0.000292,0.000174,0.000134,0.000219,0.000214
QQQ,0.000235,0.000174,0.000170,0.000135,0.000251,0.000193
VTI,0.000180,0.000134,0.000135,0.000125,0.000202,0.000138
AMD,0.000399,0.000219,0.000251,0.000202,0.001247,0.000274
AMZN,0.000266,0.000214,0.000193,0.000138,0.000274,0.000430


In [122]:
## Create Equally weighted portfolio

portfolio_value=1000000
weights=np.array([1/len(tickers)]*len(tickers))
weights

array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667])

In [123]:
portfolio_expected_return=expected_return(weights,log_returns)
portfolio_standard_deviation=standard_deviation(weights,cov_matrix)

In [124]:
portfolio_standard_deviation

0.0166627391905645

In [125]:
def rand_z_score():
    return np.random.normal(0,1)

In [126]:
days=20

def scenario_loss(portfolio_value,portfolio_standard_deviation,z_score,days):
    return portfolio_value*portfolio_expected_return*days+portfolio_value*portfolio_standard_deviation*z_score*np.sqrt(days)

In [127]:
##Simulation

n_iter=10000
scenario_return=[]

for i in range(n_iter):
    z_score=rand_z_score()
    scenario_return.append(scenario_loss(portfolio_value,portfolio_standard_deviation,z_score,days))

In [128]:
conf_intv=0.95
VaR=np.percentile(scenario_return,5)

In [129]:
VaR

-106272.09525042716

In [130]:
weights=np.random.rand(len(tickers))
weights=weights/np.sum(weights)
sum(weights)

0.9999999999999999

In [131]:
portfolio_expected_return=expected_return(weights,log_returns)
portfolio_standard_deviation=standard_deviation(weights,cov_matrix)
days=20

def scenario_loss(portfolio_value,portfolio_standard_deviation,z_score,days):
    return portfolio_value*portfolio_expected_return*days+portfolio_value*portfolio_standard_deviation*z_score*np.sqrt(days)

In [132]:
weights.shape

(6,)

In [133]:
##Simulation

n_iter=10000
scenario_return=[]

for i in range(n_iter):
    z_score=rand_z_score()
    scenario_return.append(scenario_loss(portfolio_value,portfolio_standard_deviation,z_score,days))

In [134]:
conf_intv=0.95
VaR=np.percentile(scenario_return,5)
VaR

-130322.34366072073